In [ ]:
import numpy as np
import pandas as pd



In [ ]:
df_train = pd.read_csv("/kaggle/input/playground-series-s5e3/train.csv")
df_train = df_train.drop("id", axis=1)
df_test = pd.read_csv("/kaggle/input/playground-series-s5e3/test.csv")
df_test = df_test.drop("id", axis=1)
df_test["rainfall"] = 0
submission = pd.read_csv("/kaggle/input/playground-series-s5e3/sample_submission.csv")

In [ ]:
df = pd.concat([df_train, df_test],axis=0)

In [ ]:
df_null = pd.concat([df_train, df_test], axis=0)
for i in range(len(df_null.columns)):
    print("{} columns".format(df_null.columns[i]))
    print("{} amounts without a calue or no value".format(df_null.isnull().sum(axis=0)[i]))

In [ ]:
df["winddirection"] = df["winddirection"].fillna(df["winddirection"].mean())

In [ ]:

for i in range(len(df.columns)):
    print("{} columns".format(df.columns[i]))
    print("{} amounts without a calue or no value".format(df.isnull().sum(axis=0)[i]))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df.head()

In [ ]:
train_data = df[:len(df_train)]
test_data = df[len(df_train):]

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(train_data[["day", "pressure", "maxtemp", "temparature", "mintemp", "dewpoint", "humidity", "cloud", "sunshine", "winddirection","windspeed"]])
train_data.loc[:, ["day", "pressure", "maxtemp", "temparature", "mintemp", "dewpoint", "humidity", "cloud", "sunshine", "winddirection", "windspeed"]] = scaler.transform(train_data[["day", "pressure", "maxtemp", "temparature", "mintemp", "dewpoint", "humidity", "cloud", "sunshine", "winddirection", "windspeed"]])


In [ ]:
test_data.loc[:, ["day", "pressure", "maxtemp", "temparature", "mintemp", "dewpoint", "humidity", "cloud", "sunshine", "winddirection", "windspeed"]] = scaler.transform(test_data[["day", "pressure", "maxtemp", "temparature", "mintemp", "dewpoint", "humidity", "cloud", "sunshine", "winddirection", "windspeed"]])


In [ ]:
submission["rainfall"] = 0.5
submission

In [ ]:
submission.to_csv("dummy_sub.csv", index=False)

In [ ]:
test_data = test_data.drop("rainfall", axis=1)
test_data

In [ ]:
X = train_data.drop("rainfall", axis=1)
y = train_data["rainfall"]

In [ ]:
X.shape, y.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
model_1 = LogisticRegression(max_iter=1000)
model_1.fit(X,y)

In [ ]:
pred_1 = model_1.predict(test_data)
pred_1 

In [ ]:
submission["rainfall"] = pred_1
submission

In [ ]:
submission.to_csv("LogisticRegression.csv", index=False)

In [ ]:
import lightgbm as gbm

In [ ]:
X.to_numpy()[1]

In [ ]:
gbm_train = gbm.Dataset(data=X.to_numpy(), label=y.to_numpy().ravel())
params = {"objective": "binary", "boosting": "gbdt"}

In [ ]:
#model_2 = gbm.train(params, train_set =gbm_train,num_boost_round=1000)

In [ ]:
#pred_2 = model_2.predict(test_data.to_numpy(), num_iteration=model_2.best_iteration)

In [ ]:
submission.to_csv("LightGBM.csv", index=False)

In [ ]:
import tensorflow as tf

In [ ]:
model_3 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X.shape[1], 1)),
    tf.keras.layers.LSTM(16, return_sequences = True),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.LSTM(64, return_sequences = True)   ,
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.LSTM(256, return_sequences = True),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model_3.summary()

In [ ]:
model_3.compile(optimizer = tf.keras.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0005, global_clipnorm = 1.0),
               loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
                metrics = ["accuracy" ,tf.keras.metrics.AUC()]
)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="auc",mode="max",  patience=40, restore_best_weights = True)

In [ ]:
history = model_3.fit(np.expand_dims(X.values, axis=-1), y.values, epochs=1000, batch_size=32, callbacks=[early_stopping])

In [ ]:
pred_3 = model_3.predict(test_data)
submission["rainfall"] = pred_3
submission.to_csv("submission.csv", index = False)

In [ ]:
submission